# scrape_yahoo_VOO.ipynb

2020.07.10

In [153]:
from lxml import html  
import requests
# from exceptions import ValueError
from time import sleep
import json
import argparse
from collections import OrderedDict
# from time import sleep
import time
import pandas as pd
import urllib3
urllib3.disable_warnings()

In [185]:
def parse_etf(ticker):
    url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
    url ="https://finance.yahoo.com/quote/%s?p=%s&.tsrc=fin-srch"%(ticker,ticker)
    response = requests.get(url, verify=False)
#     print( "Parsing %s"%(url))
#     time.sleep(4)
    parser = html.fromstring(response.text)
    summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    tt = summary_json_response.json()
    
    expRatio = get_ExpRatio(response)
    
    res = [ticker,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
#     tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
    str(round(expRatio*100,2)) +"%"       ,
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
    tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt'] 
          ]
    return res

def get_ExpRatio(response):
    s = response.text
    s2 = "annualReportExpenseRatio"  #239750
    s2 = "feesExpensesInvestment"    #240971

    n1=s.find(s2)
    # print(n1)
    N=400
    n2=s[n1:(n1+N)].find("annualReportExpenseRatio")
    expr = float(s[n1+n2:(n1+n2+N)].split(":")[2].split(",")[0])
    return expr

In [186]:
# !dir

In [187]:
columns = ['ticker', 'fundFamily', 'category', 'yield', 'annualReportExpenseRatio',  'ytdReturn','beta3Year' ]
tickers = ['VOO', 'CLTL','SHV','VGSH','VGIT','GOVT','IEF','BND', 'JNK', 'ARKW', 'VGT','QQQ','VDE','EDV', 'BLV', 
           'VGLT', 'VMBS',  'VTIP',  'VCIT', 'VCSH', 'VTC', 'VTEB', 
           'VIG',  'VUG', 'VYM', 'BNDW']

res = [parse_etf(ticker=tkr) for tkr in tickers ]
df=pd.DataFrame(res, columns=columns)
# df

In [213]:
type(df['yield'][0])
# df.head(n=3)
df['cy'] = [float(df['yield'][k][:-1]) - float(df['annualReportExpenseRatio'][k][:-1]) for k in range(df.shape[0])]
df['ncy'] = [round(df['cy'][k]/float(df['beta3Year'][k]),2) if float(df['beta3Year'][k])>0 else -100 for k in range(df.shape[0])]

df = df.sort_values(by=['ncy'],ascending=False).reset_index()
df=df.drop(['index'], axis=1)

In [ ]:
df.to_excel('Vang_ETF_comp.xlsx')

# BELOW IS OLD CODE

not very useful

In [ ]:
print(ticker,
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt']
)

In [120]:
other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CfeesExpensesInvestment%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
summary_json_response = requests.get(other_details_json_link)
tt = summary_json_response.json()
# feesExpensesInvestment
tt['quoteSummary']['result']

[{'defaultKeyStatistics': {'maxAge': 1,
   'priceHint': {'raw': 2, 'fmt': '2', 'longFmt': '2'},
   'enterpriseValue': {},
   'forwardPE': {},
   'profitMargins': {},
   'morningStarOverallRating': {},
   'morningStarRiskRating': {},
   'category': 'Large Blend',
   'bookValue': {},
   'priceToBook': {},
   'annualReportExpenseRatio': {},
   'ytdReturn': {'raw': -0.0145000005, 'fmt': '-1.45%'},
   'beta3Year': {'raw': 1.0, 'fmt': '1.00'},
   'totalAssets': {'raw': 533811101696,
    'fmt': '533.81B',
    'longFmt': '533,811,101,696'},
   'yield': {'raw': 0.0188, 'fmt': '1.88%'},
   'fundFamily': 'Vanguard',
   'fundInceptionDate': {'raw': 1283817600, 'fmt': '2010-09-07'},
   'legalType': 'Exchange Traded Fund',
   'threeYearAverageReturn': {'raw': 0.11270001, 'fmt': '11.27%'},
   'fiveYearAverageReturn': {'raw': 0.1118, 'fmt': '11.18%'},
   'priceToSalesTrailing12Months': {},
   'lastFiscalYearEnd': {},
   'nextFiscalYearEnd': {},
   'mostRecentQuarter': {},
   'earningsQuarterlyGrowth':

In [114]:
summary_json_response = requests.get(other_details_json_link)
tt = summary_json_response.json()
res = [ticker,
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt'] 
      ]
res

['VOO', 'Vanguard', 'Large Blend', '1.88%', {}, '-1.45%', '1.00']

In [144]:
# s = "Happy Birthday"
# s2 = "py"


In [165]:
url="https://investor.vanguard.com/etf/list#/etf/asset-class/month-end-returns"
response = requests.get(url, verify=False)

In [166]:
parser = html.fromstring(response.text)

In [168]:
parser.text

'\r\n\t'

In [ ]:
summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
    summary_json_response = requests.get(other_details_json_link)
    tt = summary_json_response.json()

# Another Query Format

In [104]:
url ="https://finance.yahoo.com/quote/%s?p=%s&.tsrc=fin-srch"%(ticker,ticker)
response = requests.get(url, verify=False)
print( "Parsing %s"%(url))
#     time.sleep(4)
parser = html.fromstring(response.text)
summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
summary_data = OrderedDict()
other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
summary_json_response = requests.get(other_details_json_link)
tt = summary_json_response.json()
# tt

C:\Users\Farid Khafizov\anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Parsing https://finance.yahoo.com/quote/VOO?p=VOO&.tsrc=fin-srch


In [215]:
response.text[:100]


# summary_table[0] #<Element tr at 0xef88a0>
# parser #<Element html at 0xed9ff60>
# summary_data #OrderedDict()

'<!doctype html>\r\n<html lang="en" id="psApp" data-modules="patterns.organisms.vanguard-products.run">'

In [103]:
print(ticker,
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt']
)

VOO Vanguard Large Blend 1.88% {} -1.45% 1.00


In [70]:
ticker="VOO"
url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
response = requests.get(url, verify=False)
print( "Parsing %s"%(url))
time.sleep(4)
parser = html.fromstring(response.text)
summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
summary_data = OrderedDict()
other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
summary_json_response = requests.get(other_details_json_link)


C:\Users\Farid Khafizov\anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Parsing http://finance.yahoo.com/quote/VOO?p=VOO


In [71]:
import json
tt = summary_json_response.json()        #['defaultKeyStatistics']
type(tt)
tt.keys()

dict_keys(['quoteSummary'])

In [219]:
# tt['quoteSummary']

In [74]:
tt['quoteSummary']['result'][0].keys()

dict_keys(['defaultKeyStatistics', 'summaryProfile'])

In [75]:
tt['quoteSummary']['result'][0]['defaultKeyStatistics'].keys()

dict_keys(['maxAge', 'priceHint', 'enterpriseValue', 'forwardPE', 'profitMargins', 'morningStarOverallRating', 'morningStarRiskRating', 'category', 'bookValue', 'priceToBook', 'annualReportExpenseRatio', 'ytdReturn', 'beta3Year', 'totalAssets', 'yield', 'fundFamily', 'fundInceptionDate', 'legalType', 'threeYearAverageReturn', 'fiveYearAverageReturn', 'priceToSalesTrailing12Months', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 'earningsQuarterlyGrowth', 'revenueQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'pegRatio', 'lastSplitFactor', 'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 'lastDividendValue', 'lastCapGain', 'annualHoldingsTurnover'])

In [84]:
print(ticker,
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['fundFamily'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['category'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['annualReportExpenseRatio'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['ytdReturn']['fmt'],
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['beta3Year']['fmt']
)


VOO Vanguard Large Blend 1.88% {} -0.92% 1.00


In [85]:
tt['quoteSummary']['result'][0]['summaryProfile'].keys()

dict_keys(['phone', 'longBusinessSummary', 'companyOfficers', 'maxAge'])

# WORKS FOR IBM and APPL

In [ ]:
from lxml import html  
import requests
# from exceptions import ValueError
from time import sleep
import json
import argparse
from collections import OrderedDict
# from time import sleep
import time

def parse(ticker):
	url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
	response = requests.get(url, verify=False)
	print( "Parsing %s"%(url))
	time.sleep(4)
	parser = html.fromstring(response.text)
	summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
	summary_data = OrderedDict()
	other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
	summary_json_response = requests.get(other_details_json_link)
	try:
		json_loaded_summary =  json.loads(summary_json_response.text)
		y_Target_Est = json_loaded_summary["quoteSummary"]["result"][0]["financialData"]["targetMeanPrice"]['raw']
		earnings_list = json_loaded_summary["quoteSummary"]["result"][0]["calendarEvents"]['earnings']
		eps = json_loaded_summary["quoteSummary"]["result"][0]["defaultKeyStatistics"]["trailingEps"]['raw']
		datelist = []
		for i in earnings_list['earningsDate']:
			datelist.append(i['fmt'])
		earnings_date = ' to '.join(datelist)
		for table_data in summary_table:
			raw_table_key = table_data.xpath('.//td[@class="C(black)"]//text()')
			raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
			table_key = ''.join(raw_table_key).strip()
			table_value = ''.join(raw_table_value).strip()
			summary_data.update({table_key:table_value})
		summary_data.update({'1y Target Est':y_Target_Est,'EPS (TTM)':eps,'Earnings Date':earnings_date,'ticker':ticker,'url':url})
		return summary_data
	except ValueError:
		print ("Failed to parse json response")
		return {"error":"Failed to parse json response"}
		


In [68]:
# parse('VOO')
# parse('VOO')


In [48]:
# if __name__=="__main__":
# 	argparser = argparse.ArgumentParser()
# 	argparser.add_argument('ticker',help = '')
# 	args = argparser.parse_args()
# 	ticker = args.ticker
# 	print( "Fetching data for %s"%(ticker))
# 	scraped_data = parse(ticker)
# 	print("Writing data to output file")
# 	with open('%s-summary.json'%(ticker),'w') as fp:
# 	 	json.dump(scraped_data,fp,indent = 4)

In [49]:
ticker='AAPL'
url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
response = requests.get(url, verify=False)
print( "Parsing %s"%(url))
time.sleep(4)
parser = html.fromstring(response.text)
summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
summary_data = OrderedDict()
other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
summary_json_response = requests.get(other_details_json_link)
summary_json_response

C:\Users\Farid Khafizov\anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Parsing http://finance.yahoo.com/quote/AAPL?p=AAPL


<Response [200]>

In [216]:
# summary_json_response.text

In [51]:
type(summary_json_response) 

requests.models.Response

In [217]:
# summary_json_response.json()

In [62]:
import json
tt = summary_json_response.json()        #['defaultKeyStatistics']
type(tt)
tt.keys()

dict_keys(['quoteSummary'])

In [63]:
tt['quoteSummary'].keys()

dict_keys(['result', 'error'])

In [64]:
tt['quoteSummary']['result'][0].keys()

dict_keys(['defaultKeyStatistics', 'summaryProfile'])

In [218]:
# tt['quoteSummary']['result'][0]['defaultKeyStatistics']

In [67]:
tt['quoteSummary']['result'][0]['defaultKeyStatistics']['yield']

{'raw': 0.0188, 'fmt': '1.88%'}